Ниже процедура для получение реквизитов из WikiData

In [66]:
import requests

def getPersonData(humanName):
    url = 'https://query.wikidata.org/sparql'
    query="""
    PREFIX wdno: <http://www.wikidata.org/prop/novalue/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?human ?humanLabel ?humanDescription ?BirthDate ?StartDate ?article WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ru". }
      ?human wdt:P31 wd:Q5 .       #find humans
      ?human ?Label "_humanName"@ru.
      OPTIONAL { ?human wdt:P569 ?BirthDate. }
      OPTIONAL { ?human wdt:P2031 ?StartDate. }
      
      ?article schema:about ?human .
      ?article schema:isPartOf <https://ru.wikipedia.org/>.

    }"""

    r = requests.get(url, params = {'format': "json", 'query': query.replace("_humanName", humanName)})
    return r.json()

data=getPersonData("Мусоргский")
print(data)


{'head': {'vars': ['human', 'humanLabel', 'humanDescription', 'BirthDate', 'StartDate', 'article']}, 'results': {'bindings': [{'human': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q132682'}, 'article': {'type': 'uri', 'value': 'https://ru.wikipedia.org/wiki/%D0%9C%D1%83%D1%81%D0%BE%D1%80%D0%B3%D1%81%D0%BA%D0%B8%D0%B9,_%D0%9C%D0%BE%D0%B4%D0%B5%D1%81%D1%82_%D0%9F%D0%B5%D1%82%D1%80%D0%BE%D0%B2%D0%B8%D1%87'}, 'BirthDate': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1839-03-09T00:00:00Z'}, 'humanLabel': {'xml:lang': 'ru', 'type': 'literal', 'value': 'Модест Петрович Мусоргский'}, 'humanDescription': {'xml:lang': 'ru', 'type': 'literal', 'value': 'русский композитор'}}]}}


Далее -- идем по директориии и сканируем файлы

In [3]:
import os

startDir = "/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ"

# r=root, d=directories, f = files
def crawlFiles(walkDir):
    for r, d, f in os.walk(walkDir):
        for file in f:
            if ".d" in file:
                yield((r, file))
                
    
for fName in crawlFiles(startDir):
    print(fName)

('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Биографическая библиотека Ф. Павленкова', 'Лаппас и Эйлер. Их жизнь и научная деятельность. (Е.Ф. Литвинова).djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Биографическая библиотека Ф. Павленкова', 'Будда. Конфуций. Савонарола. Торквемада. Лойола (Сборник).djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Библиотека ЖЗЛ. Биография продолжается', 'Геннадий Зюганов (Анатолий Житнухин) - 2007.djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Библиотека ЖЗЛ. Биография продолжается', 'Громов (Игорь Цыбульский) - 2006.djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Библиотека ЖЗЛ. Биография продолжается', 'Жерар Депардьё (Виоле Бернар) - 2009.djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Каталоги ЖЗЛ', 'Каталог «ЖЗЛ». 1890-2002 (ЖЗЛ) - 2002.djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Каталоги ЖЗЛ', 'Каталог «ЖЗЛ». 1933-1985 (ЖЗЛ) - 1987.djvu')
('/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Каталоги ЖЗЛ', 'Каталог «ЖЗЛ». 19

Из имени файла выделяем фамилию

In [47]:
def getHero(fName):
    p=fName.find(". Его")
    if p==-1: p=fName.find("(")-1
    if p<=-1: return ""
    return fName[:p]

print(getHero('Александр Даргомыжский. Его жизнь и музыкальная деятельность (Сергей Базунов).fb2'))

Александр Даргомыжский


In [63]:
import pprint
import sys

def parsePersonData(b):
    b=b[0]
    r=""
    items=['humanLabel','humanDescription','BirthDate','StartDate','article']
    for i in items:
        if r!="":r+=";"
        if i in b:
            v=b[i]['value'].partition("T00:00")[0].split("-")
            if len(v)==1:
                r=r+'"'+b[i]['value']+'"'
            else:
                if len(v)==3:
                    r=r+v[2]+'.'+v[1]+'.'+v[0]
                else:
                    r=r+'"'+''.join(v)+'"'
                
        else:
            r+=""
    return r

name=getHero('Мусоргский (Алексей Карпов) - 2012.djv')
data=getPersonData(name)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(data)
b=data['results']['bindings']
if(b!=[]): print(parsePersonData(b))

    

{   'head': {   'vars': [   'human',
                            'humanLabel',
                            'humanDescription',
                            'BirthDate',
                            'StartDate',
                            'article']},
    'results': {   'bindings': [   {   'BirthDate': {   'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                                        'type': 'literal',
                                                        'value': '1839-03-09T00:00:00Z'},
                                       'article': {   'type': 'uri',
                                                      'value': 'https://ru.wikipedia.org/wiki/%D0%9C%D1%83%D1%81%D0%BE%D1%80%D0%B3%D1%81%D0%BA%D0%B8%D0%B9,_%D0%9C%D0%BE%D0%B4%D0%B5%D1%81%D1%82_%D0%9F%D0%B5%D1%82%D1%80%D0%BE%D0%B2%D0%B8%D1%87'},
                                       'human': {   'type': 'uri',
                                                    'value': 'http://www.wikidata.org/entit

In [64]:
import os
import datetime

startDir = "/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ"

# r=root, d=directories, f = files
def crawlFiles(walkDir):
    for r, d, f in os.walk(walkDir):
        if r!=walkDir: print(r)
        for file in f:
                yield((r, file))
                
def crawlPersons(startDir):    
    for fName in crawlFiles(startDir):
        name=getHero(fName[1])
        data=getPersonData(name)
        b=data['results']['bindings']
        if(b!=[]):
            yield(parsePersonData(b)+';"'+fName[0]+"/"+fName[1]+'"')
        else:
            yield('"'+name+'";;;;'+';"'+fName[0]+"/"+fName[1]+'"')
n=1
ofName="/Users/sergey_krupeninepam.com/Desktop/Герои "+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
fu=open(ofName+" utf-8.txt", "w",  encoding="utf-8")
fc=open(ofName+" mac.txt", "w",  encoding="mac_cyrillic")
for persondata in crawlPersons(startDir):
    print(persondata, file=fu)
    print(persondata, file=fc)
    if n%10==0: print(n)
    n+=1
fc.close()
fu.close()
print("Done")

/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Биографическая библиотека Ф. Павленкова
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Библиотека ЖЗЛ. Биография продолжается
210
/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Каталоги ЖЗЛ
220
/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Библиотека ЖЗЛ Максима Горького
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Библиотека ЖЗЛ. Малая серия
1130
1140
1150
1160
/Users/sergey_krupeninepam.com/Desktop/ЖЗЛ/Периодика ЖЗЛ
Done
